In [84]:
from dotenv import load_dotenv
import os
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import TextSplitter, RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores.chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from datetime import datetime
import json
import PyPDF2
import chromadb
chroma_client = chromadb.Client()

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
DIRECTORY_PATH='/Users/matansharon/python/chat_with_doc/AI_Apps/chat_with_pdf/data'





def get_documents_names():
    documents_names = os.listdir(DIRECTORY_PATH)
    res=[]
    for doc in documents_names:
        if doc.endswith(".pdf"):
            res.append(doc)
    return res
def load_all_docs_in_data_folder(documents_names):
    
    data_documents= []
    for doc in documents_names:
        path=os.path.join(DIRECTORY_PATH+'/',doc)
        Document=PyPDF2.PdfReader(path)
        text = ''
        for page in Document.pages:
            text += page.extract_text()
        data_documents.append(text)
        
    return data_documents
    
    

def split_text(text:str):
    print("in split")
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True
    )
    chunks=text_splitter.split_text(text)
    return chunks

def create_new_db(chunks):
    print("in create db")
    
    path='/Users/matansharon/python/chat_with_doc/AI_Apps/chroma_db'
    
    if not os.path.exists(path):
        
        db=Chroma.from_texts(texts=[''],embedding=OpenAIEmbeddings(model='text-embedding-3-small'),persist_directory=path)
        return db
    if chunks:
        
        db=Chroma.from_texts(texts=chunks,embedding=OpenAIEmbeddings(model='text-embedding-3-small'),persist_directory=path)
        return db
    return load_db()

def load_db():
    
    db = Chroma(persist_directory='/Users/matansharon/python/chat_with_doc/AI_Apps/chroma_db',embedding_function=OpenAIEmbeddings(model='text-embedding-3-small'))
    return db

def get_results_with_scores(query,db):
    print("in get results with scores")
    res=db.similarity_search_with_relevance_scores(query,k=3)
    value_results=[]
    
    for i in res:
        score=i[1]
        if score>0 and score<0.5:
            value_results.append(i)
    return value_results
def get_prompt_template(results,query):
    if len(results)==0:
        # template="""
        # start the answer with the sentence: "i don't have any context to rely on so i give you the answer base on my previous knowledge."
        # and then add you answer for this {query}
        # """
        # return template
        return query
    template="""
    answer the question base only on the following context:
    {context}
    answer the question base on the above context: {query}
    
    """
    context_texts = []
    for i in range(len(results)):
        context_texts.append(results[i][0].page_content)
    temp = "\n\n---\n\n".join(context_texts)
    prompt_tamplate=ChatPromptTemplate.from_template(template)
    res=prompt_tamplate.format(context=temp,query=query)
    return res

def get_response(query,db,model):
    results=get_results_with_scores(query,db)
    prompt_template=get_prompt_template(results,query)
    response=model.invoke(prompt_template)
    return response.content
def add_document(document,db):
    
    text=""
    for page in document.pages:
        text+=page.extract_text()
    chunks=split_text(text)
    db.add_texts(chunks)
    
def main_app():
    print("in main app")
    documents_names=get_documents_names()
    docs=load_all_docs_in_data_folder(documents_names=documents_names)
    splited_docs=[]
    for doc in docs:
        chunks=split_text(doc)
        for chunk in chunks:
            splited_docs.append(chunk)
            
    print(len(splited_docs))
    db=create_new_db(splited_docs)
    # model=ChatOpenAI(model='gpt-4-turbo-preview')
    model=ChatOpenAI()
    
    return db,model,documents_names



splited_docs 101
here


/Users/matansharon/python/chat_with_docs/.venv/lib/python3.10/site-packages/langchain_core/vectorstores.py:331: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='benchmark performance does not imply strong Vicuna chatbot benchmark performance and vice\nversa—in other words, dataset suitability matters more than size for a given task.\nFurthermore, we also provide a extensive analysis of chatbot performance that uses both human\nraters and GPT-4 for evaluation. We use tournament-style benchmarking where models compete\nagainst each other in matches to produce the best response for a given prompt. The winner of a\nmatch is judged by either GPT-4 or human annotators. The tournament results are aggregated into\nElo scores [ 16,17] which determine the ranking of chatbot performance. We find that GPT-4 and\nhuman evaluations largely agree on the rank of model performance in the tournaments, but we also\nfind there are instances of strong disagreement. As suc

"Lionel Messi is a professional footballer from Argentina who is widely considered one of the greatest players of all time. He has won numerous awards and accolades throughout his career, including multiple FIFA Ballon d'Or titles. Messi has played for FC Barcelona for the majority of his career before moving to Paris Saint-Germain in 2021."

In [47]:

temp=get_prompt_template(get_results_with_scores('what is Finetuning?',db),'what is Finetuning?')
temp

'Human: \n    answer the question base only on the following context:\n    Our QLORAfinetuning method is the first method that enables the finetuning of 33B parameter\nmodels on a single consumer GPU and 65B parameter models on a single professional GPU, while\nnot degrading performance relative to a full finetuning baseline. We have demonstrated that our\nbest 33B model trained on the Open Assistant dataset can rival ChatGPT on the Vicuna benchmark.\nSince instruction finetuning is an essential tool to transform raw pretrained LLMs into ChatGPT-like\nchatbots, we believe that our method will make finetuning widespread and common in particular for\nthe researchers that have the least resources, a big win for the accessibility of state of the art NLP\ntechnology. QLORAcan be seen as an equalizing factor that helps to close the resource gap between\nlarge corporations and small teams with consumer GPUs.\nAnother potential source of impact is deployment to mobile phones. We believe our QL

In [70]:
get_results_with_scores("what is qlora?",db)

TypeError: list indices must be integers or slices, not tuple

In [51]:
model.invoke(temp)

AIMessage(content='Finetuning is the process of adjusting and optimizing a pre-trained language model by further training it on a specific dataset or task, in order to improve its performance on that particular task. In the context provided, finetuning is used to transform raw pretrained large language models (LLMs) into chatbot models like ChatGPT, with the goal of achieving state-of-the-art natural language processing (NLP) technology.')

In [52]:
model2=ChatOpenAI()
model2.invoke("what is qlora?")

AIMessage(content='There is no widely known or established meaning for the term "qlora." It is possible that it could be a unique or made-up word with no specific definition. If you have more context or information about the term, please provide it so that a more accurate answer can be given.')

In [78]:
get_response("who is messi?",db,model)

/Users/matansharon/python/chat_with_docs/.venv/lib/python3.10/site-packages/langchain_core/vectorstores.py:331: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='pairs of models compete to produce the best response for a given prompt. This is similar to how Bai\net al. [4]and Chiang et al. [10] compare models, but we also employ GPT-4 ratings in addition to\nhuman ratings. We randomly sample from the set of labeled comparisons to compute Elo [ 16,17].\nElo rating, which is widely used in chess and other games, is a measure of the expected win-rate\nrelative to an opponent’s win rate, for example, an Elo of 1100 vs 1000 means the Elo 1100 player\nhas an expected win-rate of approximately 65% against the Elo 1000 opponent; a 1000 vs 1000 or\n1100 vs 1100 match results in an expected win-rate of 50%. The Elo rating changes after each match\nproportionally to the expected outcome, that is, an unexpected upset leads to a large change in Elo\nrating while an

"I don't have any context to rely on so I give you the answer based on my previous knowledge. In situations like this, it's important to gather more information before making any decisions."

In [ ]:
# len(chunks)

# model=ChatOpenAI()
# query='what is qlora?'

# response=get_response(query,db,model)
# print(response)
# db=create_new_db('')

In [ ]:
import chromadb.utils.embedding_functions as embedding_functions
import chromadb
from PyPDF2 import PdfReader
path='/Users/matansharon/python/chat_with_docs/data/pdf/qlora.pdf'
doc=PdfReader(path)
#get the name of the document
doc_name=path.split('/')[-1]

text=''
for page in doc.pages:
    text+=page.extract_text()



chunks=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100,length_function=len,add_start_index=True).split_text(text)




In [ ]:
print(get_response('who is the authors of the paper "Attention Is All You Need"',db,model))

In [ ]:
chat=ChatOpenAI(model='gpt-4-turbo-preview')


In [ ]:
print(chat.invoke(f'base on this: {content} what is qlora?'))